In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
from skimage.measure import compare_psnr,compare_mse
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
from keras.preprocessing.image import ImageDataGenerator
import scipy.io as sio
import pickle
from random import randint
from utilities import *
%matplotlib inline


Using TensorFlow backend.


In [3]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append(img)
    return np.array(images)
        

In [3]:
from PIL import Image

In [3]:
def resize_images(src_path,dest_path,sizeX,sizeY):
    images = []
    width,height = sizeX,sizeY
    for filename in os.listdir(src_path):
        img = cv2.imread(src_path + filename)
        if img is not None:
            images.append(img)
    cnt =1
    for img in images:
        im2 = cv2.resize(img,(width, height),interpolation = cv2.INTER_CUBIC)
        
        
        ext = ".png"
        cv2.imwrite(dest_path + "image" +  str(cnt) + ext,im2)
        cnt+=1

In [5]:
raw_images = read_images_from_folder('./data/oct_gt/')
raw_images = np.resize(raw_images,(raw_images.shape[0],448,900,1))

In [4]:
#Adding noise to images
def add_noise(images):
    
    noisy_set = []
    for image in images:
        print len(noisy_set)
        for shape in xrange(2,9,1):
            for scale in xrange(2,9,1):
                noise = np.random.gamma(shape,scale,image.shape)
                noisy_set.append((image,np.clip(noise + image,0.,255.)))
    shuffle(noisy_set)
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def expand_pair(noisy_set):   
    ground_truth=[]
    noisy_images = []
    for i in range(noisy_set.shape[0]):
        ground_truth.append(noisy_set[i][0].reshape((noisy_set[i][0].shape[0],noisy_set[i][0].shape[1],1)))
        #print( str(noisy_set[i][0].shape[0]) +" "+ str(noisy_set[i][0].shape[1]))
        noisy_images.append(noisy_set[i][1].reshape((noisy_set[i][1].shape[0],noisy_set[i][1].shape[1],1)))
    return np.array(ground_truth), np.array(noisy_images)

In [7]:
noisy_set = add_noise(raw_images)
for i in range(noisy_set.shape[0]):
    cv2.imwrite("./data/oct_noisy/"+str(i)+".tif",noisy_set[i][1])    

0
49
98
147
196
245
294
343
392
441
490
539
588
637
686
735
784


In [ ]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = expand_pair(noisy_set)
print (ground_truth.shape, noisy_images.shape, noisy_set.shape)

((833, 448, 900, 1), (833, 448, 900, 1), (833, 2, 448, 900, 1))


In [ ]:
print np.std(ground_truth/255.)

In [5]:
def resize_images(images,size):
    resized_images = []
    
    for img in images:
        im2 = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
        resized_images.append(im2)
    
    resized_images = np.array(resized_images)
    resized_images = np.reshape(resized_images,(resized_images.shape[0],resized_images.shape[1],resized_images.shape[2],1))
    
    return resized_images

ground_truth_rs = resize_images(ground_truth,(256,128))
noisy_images_rs = resize_images(noisy_images,(256,128))

NameError: name 'ground_truth' is not defined

In [11]:
noisy_images_rs.shape

(833, 128, 256, 1)

In [12]:
for i in range(noisy_images_rs.shape[0]):
    cv2.imwrite("./data/oct_noisy_resized/"+str(i)+".tif",noisy_images_rs[i])  
    
for i in range(ground_truth_rs.shape[0]):
    cv2.imwrite("./data/oct_gt_resized/"+str(i)+".tif",ground_truth_rs[i])    

In [4]:
ground_truth_rs = read_images_from_folder('./data/oct_gt_resized/')
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))

noisy_images_rs = read_images_from_folder('./data/oct_noisy_resized/')
noisy_images_rs = np.resize(noisy_images_rs,(noisy_images_rs.shape[0],noisy_images_rs.shape[1],noisy_images_rs.shape[2],1))

In [7]:
cv2.imwrite('test_noisy.png',noisy_images_rs[33])
cv2.imwrite('test.png',ground_truth_rs[33])

True

In [5]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth_rs.shape[0]*0.8)
x_train = ground_truth_rs[0:train_size]/255.
x_train_noisy = noisy_images_rs[0:train_size]/255.
x_test = ground_truth_rs[train_size:]/255.
x_test_noisy = noisy_images_rs[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(666, 128, 256, 1)
(167, 128, 256, 1)


In [32]:
#for i in range(ground_truth.shape[0]):
 #   cv2.imwrite("./data/cifar-bw/image"+str(i)+".png",ground_truth[i])
    
#for i in range(noisy_images.shape[0]):
 #   cv2.imwrite("./data/noisy-cifar-bw/image"+str(i)+".png",noisy_images[i])

In [6]:
#lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
#csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
#early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [7]:
x_train_noisy.shape

(666, 128, 256, 1)

In [10]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=((None,None,1)))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        
        
        autoencoder.compile(optimizer='Adam', loss='mean_squared_error',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [11]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()
#plot_model(autoencoder, to_file='model.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, None, None, 64)    0         
__________

In [12]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

Train on 666 samples, validate on 167 samples
Epoch 1/500
 80/666 [==>...........................] - ETA: 3:04 - loss: 0.0351 - acc: 0.0000e+00 - mean_squared_error: 0.0351

KeyboardInterrupt: 

In [ ]:
#Perceptual loss


In [10]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy','mean_squared_error'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [11]:
sym_autoencoder = get_cnn_dsc_architecture()
plot_model(autoencoder, to_file='sys.png')

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Image_input (InputLayer)         (None, None, None, 1) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 64 1088        Image_input[0][0]                
____________________________________________________________________________________________________
input_1 (InputLayer)             (1,)                  0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, None, None, 64 0           conv2d_1[0][0]                   
                                                                   input_1[0]

NameError: name 'autoencoder' is not defined

In [12]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [ ]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=8,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 666 samples, validate on 167 samples
Epoch 1/500
666/666 [==============================] - 181s - loss: 0.0351 - acc: 0.0000e+00 - mean_squared_error: 0.0351 - val_loss: 0.0064 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0064
Epoch 2/500
666/666 [==============================] - 168s - loss: 0.0055 - acc: 0.0000e+00 - mean_squared_error: 0.0055 - val_loss: 0.0058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0058
Epoch 3/500
666/666 [==============================] - 168s - loss: 0.0051 - acc: 0.0000e+00 - mean_squared_error: 0.0051 - val_loss: 0.0056 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0056
Epoch 4/500
666/666 [==============================] - 168s - loss: 0.0048 - acc: 0.0000e+00 - mean_squared_error: 0.0048 - val_loss: 0.0053 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0053
Epoch 5/500
666/666 [==============================] - 168s - loss: 0.0046 - acc: 0.0000e+00 - mean_squared_error: 0.0046 - val_loss: 0.0055 - val_acc: 0.0000e+00 - val_m

666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0038 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0038
Epoch 41/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0039 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0039
Epoch 42/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0039 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0039
Epoch 43/500
666/666 [==============================] - 168s - loss: 0.0031 - acc: 0.0000e+00 - mean_squared_error: 0.0031 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 44/500
666/666 [==============================] - 168s - loss: 0.0030 - acc: 0.0000e+00 - mean_squared_error: 0.0030 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 45/500
666/666 [======

666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0030 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0030
Epoch 82/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0030 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0030
Epoch 83/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0036 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0036
Epoch 84/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 85/500
666/666 [==============================] - 168s - loss: 0.0025 - acc: 0.0000e+00 - mean_squared_error: 0.0025 - val_loss: 0.0032 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0032
Epoch 86/500
666/666 [======

666/666 [==============================] - 168s - loss: 0.0024 - acc: 0.0000e+00 - mean_squared_error: 0.0024 - val_loss: 0.0041 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0041
Epoch 104/500
666/666 [==============================] - 168s - loss: 0.0024 - acc: 0.0000e+00 - mean_squared_error: 0.0024 - val_loss: 0.0028 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0028
Epoch 105/500
666/666 [==============================] - 168s - loss: 0.0023 - acc: 0.0000e+00 - mean_squared_error: 0.0023 - val_loss: 0.0027 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0027
Epoch 106/500
666/666 [==============================] - 168s - loss: 0.0023 - acc: 0.0000e+00 - mean_squared_error: 0.0023 - val_loss: 0.0032 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0032
Epoch 107/500
666/666 [==============================] - 168s - loss: 0.0023 - acc: 0.0000e+00 - mean_squared_error: 0.0023 - val_loss: 0.0027 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0027
Epoch 108/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0022 - acc: 0.0000e+00 - mean_squared_error: 0.0022 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 148/500
666/666 [==============================] - 168s - loss: 0.0022 - acc: 0.0000e+00 - mean_squared_error: 0.0022 - val_loss: 0.0025 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0025
Epoch 149/500
666/666 [==============================] - 168s - loss: 0.0021 - acc: 0.0000e+00 - mean_squared_error: 0.0021 - val_loss: 0.0032 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0032
Epoch 150/500
666/666 [==============================] - 168s - loss: 0.0021 - acc: 0.0000e+00 - mean_squared_error: 0.0021 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 151/500
666/666 [==============================] - 168s - loss: 0.0021 - acc: 0.0000e+00 - mean_squared_error: 0.0021 - val_loss: 0.0030 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0030
Epoch 152/500
666/666 [=

...

666/666 [==============================] - 168s - loss: 0.0019 - acc: 0.0000e+00 - mean_squared_error: 0.0019 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 239/500
666/666 [==============================] - 168s - loss: 0.0019 - acc: 0.0000e+00 - mean_squared_error: 0.0019 - val_loss: 0.0023 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0023
Epoch 240/500
666/666 [==============================] - 168s - loss: 0.0019 - acc: 0.0000e+00 - mean_squared_error: 0.0019 - val_loss: 0.0027 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0027
Epoch 241/500
666/666 [==============================] - 168s - loss: 0.0019 - acc: 0.0000e+00 - mean_squared_error: 0.0019 - val_loss: 0.0021 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0021
Epoch 242/500
666/666 [==============================] - 168s - loss: 0.0019 - acc: 0.0000e+00 - mean_squared_error: 0.0019 - val_loss: 0.0021 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0021
Epoch 243/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0020 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0020
Epoch 285/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0020 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0020
Epoch 286/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0021 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0021
Epoch 287/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0020 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0020
Epoch 288/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 289/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 330/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0019 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0019
Epoch 331/500
666/666 [==============================] - 168s - loss: 0.0018 - acc: 0.0000e+00 - mean_squared_error: 0.0018 - val_loss: 0.0029 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0029
Epoch 332/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 333/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0025 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0025
Epoch 334/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0018 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0018
Epoch 377/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 378/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0021 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0021
Epoch 379/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 380/500
666/666 [==============================] - 168s - loss: 0.0017 - acc: 0.0000e+00 - mean_squared_error: 0.0017 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 381/500
666/666 [=

666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0028 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0028
Epoch 400/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 401/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0018 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0018
Epoch 402/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0024 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0024
Epoch 403/500
666/666 [==============================] - 168s - loss: 0.0016 - acc: 0.0000e+00 - mean_squared_error: 0.0016 - val_loss: 0.0022 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0022
Epoch 404/500
666/666 [=

In [21]:
#getting testing images

In [28]:
autoencoder = load_model('./models/simple_cnn_autoencoder.hdf5')

In [101]:
#plt.imshow(out_autoencoder[0])

def predict_save(model,image_path):
    test = cv2.imread(image_path,0)
    test = np.array(test)
    test = test.reshape(1,test.shape[0],test.shape[1],1)
    test =  test/255.0
    
    out = model.predict(test,verbose=1)
    cv2.imwrite("out.png",(out[0]*255))


In [103]:
predict_save(autoencoder,'./data/standard_test_images/noise/image21.png')

1/1 [==============================] - 0s


In [119]:
for i in range(100):
    cv2.imwrite("./data/denoise-cifar-bw/"+str(i)+".png",out_autoencoder[i]*255.)

In [6]:

def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(compare_mse(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += compare_psnr(x_test[i].reshape(32,32,1),out[i].reshape(32,32,1),data_range=255)
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],32,32,1)
    noisySet = noisySet.reshape(noisySet.shape[0],32,32,1)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [7]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32)
    denoised = []
    count = 1
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        if (count%10 == 0):
            print (str(count)+ "images denoised")
        count+=1
        
    return np.array(denoised)

def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],32,32,1)
    denoised = []
    count = 1
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5,multichannel = False)
        denoised.append(denoised_image)
        if(count%100 == 0) :
            print(str(count)+" images denoised")
        count+=1
    return np.array(denoised)

In [134]:
print (test_images.shape,out_autoencoder.shape)
print (compare_mse(test_images[0].flatten(),(out_autoencoder[0]*255.).flatten()))
get_psnr_result(out_autoencoder*255.,test_images)

((10000, 32, 32, 1), (10000, 32, 32, 1))
458.092258419


22.066576979897317

In [39]:
print (test_images.shape,out_sym_autoencoder.shape)
print (mean_squared_error(test_images[0].flatten(),out_sym_autoencoder[0].flatten()))
get_psnr_result(out_sym_autoencoder,test_images)

NameError: name 'out_sym_autoencoder' is not defined

In [39]:
import bm3d

In [40]:
noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

error: /home/travis/miniconda/conda-bld/work/opencv-2.4.11/modules/core/src/dxt.cpp:2330: error: (-213) Odd-size DCT's are not implemented in function dct


In [ ]:
bm3d_out_norm = bm3d_out.astype('float64')/255.0

In [ ]:
get_psnr_result(bm3d_out_norm,test_images)

In [166]:
print(noisy_test_images.shape)
nlm_out = nlm_denoise(noisy_test_images*255.)
nlm_out = nlm_out.astype('float64')

(10000, 32, 32, 1)


KeyboardInterrupt: 

In [157]:
print nlm_out.shape,test_images.shape
get_psnr_result(nlm_out,test_images)

(10000, 32, 32) (10000, 32, 32, 1)


18.414818545893922

In [136]:
get_ssim_result(test_images,out_autoencoder*255.)

0.69608629344603479

In [66]:
get_ssim_result(test_images,out_sym_autoencoder)

0.718734871276698

In [ ]:
get_ssim_result(test_images,bm3d_out_norm)

In [69]:
get_ssim_result(test_images,nlm_out)

0.49438484841716263

In [30]:
model = load_model("./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5")

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_3 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_4 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, None, None, 64)    0         
__________

In [6]:
#Testing
phantom_test = sio.loadmat('./test_data/phantom.mat')

In [21]:
np.array(phantom_test['InputImage'])

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [9]:
cv2.imwrite('./test_data/phantom.png',phantom_test['InputImage'])

True

In [26]:
def read_images_from_folder_with_names(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((filename,img))
    return images
       

In [54]:
#Adding Gamma noise
test_image = cv2.imread('./test_data/phantom.tif',0)
#test_image = cv2.resize(test_image,(900,448),interpolation = cv2.INTER_CUBIC)
print test_image.shape

gamma_param = [(3,2),(4,3),(4,4),(6,6),(8,7)]
for param in gamma_param:
    shape = param[0]
    scale = param[1]
    noise = np.random.gamma(shape,scale,test_image.shape)
    noisy_img = np.clip(noise + test_image,0.,255.)
    cv2.imwrite('./test_data/phantom_noisy/phantom_shape_'+str(shape)+'_scale_'+str(scale)+'.tif',noisy_img)

(790, 790)


In [55]:
test_image_set = read_images_from_folder_with_names('./test_data/phantom_noisy/')

In [56]:
test_image_set

[('phantom_shape_4_scale_3.tif', array([[ 6, 14, 21, ..., 12, 14,  3],
         [11,  5,  7, ..., 10,  6, 16],
         [ 7,  4,  3, ..., 14,  6,  8],
         ..., 
         [ 6, 11, 10, ..., 11, 11, 16],
         [19,  7, 18, ...,  7,  6,  7],
         [12, 15, 15, ..., 15, 18,  8]], dtype=uint8)),
 ('phantom_shape_3_scale_2.tif', array([[10,  4,  9, ...,  3,  6,  9],
         [ 2,  6,  4, ...,  5, 12,  8],
         [ 6, 17, 10, ...,  5,  8,  6],
         ..., 
         [ 4,  6, 12, ...,  5,  6,  6],
         [ 8,  9, 10, ...,  5,  3, 10],
         [ 5,  1,  5, ...,  5,  6,  3]], dtype=uint8)),
 ('phantom_shape_8_scale_7.tif', array([[ 47,  69,  48, ...,  84,  25,  43],
         [ 56,  27,  45, ...,  48,  42,  58],
         [ 53, 151,  35, ...,  46,  43,  62],
         ..., 
         [ 30,  49,  51, ...,  34,  39,  44],
         [ 66,  88,  52, ...,  46,  40,  60],
         [ 48,  55, 103, ...,  29,  86,  82]], dtype=uint8)),
 ('phantom_shape_4_scale_4.tif', array([[13, 20, 15, ..., 

In [83]:
for idx,test_image in enumerate(test_image_set):
    test_image_norm = cv2.resize(test_image[1],(792,792),interpolation = cv2.INTER_CUBIC)

    test_image_norm = [test_image_norm]
    test_image_norm = np.array(test_image_norm,dtype=np.float64)
    test_image_norm = test_image_norm/255.
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    
    out_image = model.predict(test_image_norm,verbose=1)
    out_image = out_image.astype(np.float64)
    print test_image_norm[0].shape,out_image[0].shape
    psnr = compare_psnr(test_image_norm[0]*255.,out_image[0]*255.,data_range=256)
    ssim = compare_ssim(test_image_norm[0],out_image[0],multichannel=True)
    print psnr,ssim
    print test_image[0][:-4]
    print cv2.imwrite('./test_data/phantom_denoised/'+test_image[0][:-4] + '_'+str(psnr)+'.tif',out_image[0]*255.)
    print './test_data/phantom_denoised/'+test_image[0][:-4] + '_'+str(psnr)+'.tif'


1/1 [==============================] - 3s 3s/step
(792, 792, 1) (792, 792, 1)
14.1688053995 0.538685767461
phantom_shape_4_scale_3
True
./test_data/phantom_denoised/phantom_shape_4_scale_3_14.1688053995.tif
1/1 [==============================] - 3s 3s/step
(792, 792, 1) (792, 792, 1)
13.0058097454 0.481478793641
phantom_shape_3_scale_2
True
./test_data/phantom_denoised/phantom_shape_3_scale_2_13.0058097454.tif
1/1 [==============================] - 2s 2s/step
(792, 792, 1) (792, 792, 1)
16.436941766 0.462860742425
phantom_shape_8_scale_7
True
./test_data/phantom_denoised/phantom_shape_8_scale_7_16.436941766.tif
1/1 [==============================] - 2s 2s/step
(792, 792, 1) (792, 792, 1)
14.8847776604 0.55732322882
phantom_shape_4_scale_4
True
./test_data/phantom_denoised/phantom_shape_4_scale_4_14.8847776604.tif
1/1 [==============================] - 2s 2s/step
(792, 792, 1) (792, 792, 1)
17.9598867905 0.57967343949
phantom_shape_6_scale_6
True
./test_data/phantom_denoised/phantom_sha

In [64]:
out_image = model.predict(test_image_set[1][1],verbose=1)

ValueError: Error when checking : expected input_2 to have 4 dimensions, but got array with shape (790, 790)

In [61]:
cv2.imwrite('denoise_test.tif',out_image[0]*255.)

True

In [24]:
test_image.shape

(450, 900)

In [41]:
model = autoencoder